# M52 - MGE

Monopoly with IRTS - Calibrated to zero profits

|Markets  |   X  |      N   |     Y   |     W    |  CONS|
|---------|------|----------|---------|----------|------|
|    PX   | 100  |          |         |  -100    |      | 
|    PY   |      |          |   100   |  -100    |      |
|    PF   |      |     20   |         |          |   -20|
|    PU   |      |          |         |   200    |  -200|
|    PW   | -32  |     -8   |   -60   |          |   100|
|    PZ   | -48  |    -12   |   -40   |          |   100|
|    MK   | -20  |          |         |          |    20|

In [48]:
using MPSGE_MP

In [49]:
M52 = MPSGEModel()

@parameters(M52, begin
    σ, 9, (description = "Elasticity of substitution in demand",)
    FCOST, 1, (description = "Ratio of fixed costs to benchmark",)
    ENDOW, 1, (description = "Level of factor endowment",)
end)

@sectors(M52, begin
	X,	(description = "Activity level -- monopolist sector X",)
	Y,	(description = "Activity level -- competitive sector Y",)
	W,	(description = "Welfare index for the consumer",)
end)
@commodities(M52, begin
	PU,	(description = "Welfare price index for the consumer",)
	PX,	(description = "Price index for X (gross of markup)",)
	PY,	(description = "Price index for Y (gross of markup)",)
	PW,	(description = "Price index for labor",)
	PZ,	(description = "Price index for capital",)
end)
@consumers(M52, begin
	CONS,	(description = "All consumers",)
end)
@auxiliaries(M52, begin
	SHAREX,	(description = "Value share of X in total consumption",)
	MARKUP,	(description = "Markup based on Marshallian demand",)
end)

@production(M52, X, [s=1,t=0], begin
    @output(PX, 80, t, taxes = [Tax(CONS,MARKUP)])
    @input(PW, 32, s)
    @input(PZ, 48, s)
end)

@production(M52, Y, [s=1,t=0], begin
    @output(PY, 100, t)
    @input(PW, 60, s)
    @input(PZ, 40, s)
end)

@production(M52, W, [s=σ,t=0], begin
    @output(PU, 200, t)
    @input(PX, 80, s, reference_price = 1.25)
    @input(PY, 100, s)
end)

@demand(M52, CONS,begin
    @final_demand(PU, 200)
end, begin
    @endowment(PW, 100*ENDOW - 8*FCOST)
    @endowment(PZ, 100*ENDOW - 12*FCOST)
end)

@aux_constraint(M52, SHAREX, SHAREX*(80*PX*X + 100*PY*Y) - 80*PX*X)

@aux_constraint(M52, MARKUP, MARKUP - 1/(σ - (σ-1)*SHAREX));

# Benchmark

In [50]:
set_start_value(PX, 1.25)
set_start_value(SHAREX, .5)
set_start_value(MARKUP, .2)

solve!(M52, cumulative_iteration_limit=0)
generate_report(M52)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_77C8.tmp
 > cumulative_iteration_limit 0
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris
Preprocessed size   : 10

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 0.0000e+00           I 0.0e+00 0.0e+00 (zero_profit[X)

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.000000
Residual. . . . . . . . 0.000000e+00
Postsolved residual: 0.0000e+00


Solver Status: LOCALLY_SOLVED
Model Status: FEASIBLE_POINT

Default price normalization using income for CONS - This value is fixed. Unfix with unfix(CONS).

Row,var,value,margin
,GenericV…,Float64,Float64
1,Y,1.0,0.0
2,X,1.0,0.0
3,W,1.0,0.0
4,PU,1.0,0.0
5,PW,1.0,0.0
6,PZ,1.0,0.0
7,PY,1.0,0.0
8,PX,1.25,0.0
9,CONS,200.0,0.0


In [51]:
incomeM = value(W*(MARKUP*PX*X*80 - PW*8*FCOST - PZ*12*FCOST)/(PX*X*80 + PY*Y*100))
incomeC = value(W) - incomeM

println("incomeM: $incomeM\nincomeC: $incomeC")

incomeM: 0.0
incomeC: 1.0


# Counterfactual

In [52]:
#unfix(CONS)

fix(MARKUP,0)

solve!(M52)
generate_report(M52)


Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_77F8.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 9

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             3.9025e+01             0.0e+00 (zero_profit[X)
    1     1     0     9 1.0141e+01  1.0e+00    0.0e+00 (zero_profit[W)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 1.0141e+01           I 0.0e+00 9.8e+00 (zero_profit[W)
    1     1     3     3 3.2204e-01  1.0e+00 SO 0.0e+00 2.8e-01 (market_clearanc)
    2     1     4     4 6.0479e-04  1.0e+00 SO 0.0e+00 5.6e-04 (market_clearanc)
    3     1     5     5 1.6245e-09  1.0e+00 SO 0.0e+00 1.5e-09 (market_clearanc)

Major Iterations. . . . 3
Minor Iterations. . . . 3
Rest

Row,var,value,margin
,GenericV…,Float64,Float64
1,Y,0.387179,-1.42109e-14
2,X,1.7449,0.0
3,W,1.03973,1.13687e-13
4,PU,0.961791,1.49976e-9
5,PW,0.95991,-1.90425e-12
6,PZ,1.26919,1.42109e-12
7,PY,1.07337,-2.10321e-12
8,PX,1.13503,2.21689e-12
9,CONS,200.0,1.44257e-9


In [53]:
incomeM = value(W*(MARKUP*PX*X*80 - PW*8*FCOST - PZ*12*FCOST)/(PX*X*80 + PY*Y*100))
incomeC = value(W) - incomeM

println("incomeM: $incomeM\nincomeC: $incomeC")

incomeM: -0.11909810769062674
incomeC: 1.1588248899335793


# Counterfactual

In [54]:
set_value!(ENDOW,2)

unfix(MARKUP)

solve!(M52)
generate_report(M52)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_7818.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 10

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             2.8144e+02             0.0e+00 (market_clearance[PW)
    1     2     0    10 5.6498e+01  8.0e-01    0.0e+00 (market_clearance[PZ)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     3     2 5.6498e+01           I 0.0e+00 4.3e+01 (market_clearanc)
    1     1     4     3 1.2274e+00  1.0e+00 SM 0.0e+00 8.6e-01 (market_clearanc)
    2     1     5     4 5.2696e-01  1.0e+00 SO 0.0e+00 4.2e-01 (market_clearanc)
    3     1     6     5 4.4815e-01  1.0e+00 SO 0.0e+00 4.5e-01 (market_clearanc)
    4     1     7     6 6.0004e-02  1.0e+0

Row,var,value,margin
,GenericV…,Float64,Float64
1,Y,2.09209,-1.42109e-14
2,X,2.13465,-5.32907e-15
3,W,2.11336,0.0
4,PU,0.47318,3.97904e-13
5,PW,0.477558,-5.68434e-14
6,PZ,0.468002,-1.13687e-13
7,PY,0.473712,5.68434e-14
8,PX,0.590817,-2.84217e-14
9,CONS,200.0,2.27374e-13


In [55]:
incomeM = value(W*(MARKUP*PX*X*80 - PW*8*FCOST - PZ*12*FCOST)/(PX*X*80 + PY*Y*100))
incomeC = value(W) - incomeM

println("incomeM: $incomeM\nincomeC: $incomeC")

incomeM: 0.11505239718236737
incomeC: 1.9983080227908019


# Counterfactual

In [56]:
set_value!(ENDOW,.75)

solve!(M52)
generate_report(M52)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_7858.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 10

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             1.3859e+00             0.0e+00 (market_clearance[PW)
    1     1     0    10 1.2920e+00  1.0e+00    0.0e+00 (market_clearance[PU)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 1.2920e+00           I 0.0e+00 1.2e+00 (market_clearanc)
    1     1     3     3 6.2013e-01  1.0e+00 SO 0.0e+00 6.2e-01 (market_clearanc)
    2     1     4     4 5.2325e-03  1.0e+00 SO 0.0e+00 5.2e-03 (market_clearanc)
    3     1     5     5 1.9047e-07  1.0e+00 SO 0.0e+00 1.9e-07 (market_clearanc)

Major Iterations. . . . 3
Minor Iteration

Row,var,value,margin
,GenericV…,Float64,Float64
1,Y,0.726951,-1.08514e-10
2,X,0.716268,-7.70992e-10
3,W,0.721608,-5.97993e-11
4,PU,1.38579,1.90282e-7
5,PW,1.37651,-2.87251e-9
6,PZ,1.39698,2.93151e-9
7,PY,1.38466,-5.15465e-9
8,PX,1.73367,4.14182e-9
9,CONS,200.0,2.6455e-7


In [57]:
incomeM = value(W*(MARKUP*PX*X*80 - PW*8*FCOST - PZ*12*FCOST)/(PX*X*80 + PY*Y*100))
incomeC = value(W) - incomeM

println("incomeM: $incomeM\nincomeC: $incomeC")

incomeM: -0.028905562415386773
incomeC: 0.7505132094856337
